import packages


In [1]:
import pandas as pd
import nltk
import re
from nltk.sentiment.vader import SentimentIntensityAnalyzer
nltk.download("stopwords")

from nltk.corpus import stopwords
stopwords=set(stopwords.words('english'))

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\praga\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [2]:
data=pd.read_csv('flipkart_reviews.csv')

In [4]:
data.head(10)

,Product_name,Review,Rating
0,Lenovo Ideapad Gaming 3 Ryzen 5 Hexa Core 5600...,Best under 60k Great performanceI got it for a...,5
1,Lenovo Ideapad Gaming 3 Ryzen 5 Hexa Core 5600...,Good perfomence...,5
2,Lenovo Ideapad Gaming 3 Ryzen 5 Hexa Core 5600...,Great performance but usually it has also that...,5
3,DELL Inspiron Athlon Dual Core 3050U - (4 GB/2...,My wife is so happy and best product 👌🏻😘,5
4,DELL Inspiron Athlon Dual Core 3050U - (4 GB/2...,"Light weight laptop with new amazing features,...",5
5,DELL Inspiron Athlon Dual Core 3050U - (4 GB/2...,"Amazing laptop, am so much happy, thanks for F...",5
6,DELL Inspiron Athlon Dual Core 3050U - (4 GB/2...,Over all a good laptop for personal use,5
7,DELL Inspiron Athlon Dual Core 3050U - (4 GB/2...,Thank you so much Flipkart,4
8,DELL Inspiron Athlon Dual Core 3050U - (4 GB/2...,Amazing product,5
9,DELL Inspiron Athlon Dual Core 3050U - (4 GB/2...,"Good for normal work , students, online classe...",3


In [5]:
data.isnull().sum()

Product_name    0
Review          0
Rating          0
dtype: int64

In [6]:
stemmer=nltk.SnowballStemmer("english")
def clean(text):
    text=str(text).lower()
    text=re.sub('\[.*?\]','',text)
    text=re.sub('https?://\S+|WWW\.\S+','',text)
    text=re.sub('<.*?>+','',text)
    text=re.sub('\n','',text)
    text=re.sub('\W*\d\W*','',text)
    text=[word for word in text.split(' ')]
    text=" ".join(text)
    text=[stemmer.stem(word) for word in text.split(' ')]
    text=" ".join(text)
    return text
data["Review"]=data["Review"].apply(clean)

In [7]:
ratings=data["Rating"].value_counts()
numbers=ratings.index
quantity=ratings.values
import plotly.express as px
figure=px.pie(data,values=quantity,names=numbers,hole=0.5)
figure.show()

In [8]:
nltk.download("vader_lexicon")
sentiment=SentimentIntensityAnalyzer()
data['Positive']=[sentiment.polarity_scores(i)["pos"] for i in data["Review"]]
data['Negative']=[sentiment.polarity_scores(i)["neg"] for i in data["Review"]]
data['Neutral']=[sentiment.polarity_scores(i)["neu"] for i in data["Review"]]
data=data[["Review","Positive","Negative","Neutral"]]
data.head(10)

[nltk_data] Downloading package vader_lexicon to
[nltk_data]     C:\Users\praga\AppData\Roaming\nltk_data...
[nltk_data]   Package vader_lexicon is already up-to-date!


,Review,Positive,Negative,Neutral
0,best underk great performancei got it for arou...,0.317,0.076,0.607
1,good perfomence...,0.744,0.000,0.256
2,great perform but usual it has also that game ...,0.142,0.000,0.858
3,my wife is so happi and best product 👌🏻😘,0.358,0.000,0.642
4,"light weight laptop with new amaz features, ba...",0.291,0.000,0.709
5,"amaz laptop, am so much happy, thank for flipk...",0.491,0.000,0.509
6,over all a good laptop for person use,0.326,0.000,0.674
7,thank you so much flipkart,0.385,0.000,0.615
8,amaz product,0.000,0.000,1.000
9,"good for normal work , students, onlin classes...",0.188,0.000,0.812


In [10]:
print("Positive Review",x)
print("Negative Review",y)
print("Neutral Review",z)

Positive Review 719.5750000000011
Negative Review 92.45100000000004
Neutral Review 1491.9489999999994


In [11]:
def label_sentiment(x:float):
    if x < -0.05 : return 'negative'
    if x > 0.35 : return 'positive'
    return 'neutral'


In [14]:
data['sentiment'] = [sentiment.polarity_scores(x)['compound'] for x in data['Review']]
data['overall_sentiment'] = data['sentiment'].apply(label_sentiment);

In [16]:
data.head(10)

,Review,Positive,Negative,Neutral,sentiment,overall_sentiment
0,best underk great performancei got it for arou...,0.317,0.076,0.607,0.9540,positive
1,good perfomence...,0.744,0.000,0.256,0.4404,positive
2,great perform but usual it has also that game ...,0.142,0.000,0.858,0.6858,positive
3,my wife is so happi and best product 👌🏻😘,0.358,0.000,0.642,0.6666,positive
4,"light weight laptop with new amaz features, ba...",0.291,0.000,0.709,0.6249,positive
5,"amaz laptop, am so much happy, thank for flipk...",0.491,0.000,0.509,0.7745,positive
6,over all a good laptop for person use,0.326,0.000,0.674,0.4404,positive
7,thank you so much flipkart,0.385,0.000,0.615,0.3612,positive
8,amaz product,0.000,0.000,1.000,0.0000,neutral
9,"good for normal work , students, onlin classes...",0.188,0.000,0.812,0.7003,positive


In [17]:
x=sum(data["Positive"])
y=sum(data["Negative"])
z=sum(data["Neutral"])
def sentiment_score(a,b,c):
    if(a>b) and(a>c):
        print("Positive")
    elif(b>a) and (b>c):
        print("Negative")
    else:
        print("Neutral")
sentiment_score(x,y,z)

Neutral


Hence The overall Review Of The Dataset Is Neutral